In [12]:
from google import genai

client = genai.Client()
prompt = "The quick brown fox jumps over the lazy dog."

# Count tokens using the new client method.
total_tokens = client.models.count_tokens(
    model="gemini-2.0-flash", contents=prompt
)
# print("total_tokens: ", total_tokens)
# ( e.g., total_tokens: 10 )

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt
)
print(response.text)
# The usage_metadata provides detailed token counts.
print("\nInput tokens:", response.usage_metadata.prompt_token_count)
print("Output tokens:", response.usage_metadata.candidates_token_count)
print("Total tokens:", response.usage_metadata.total_token_count)

This is a classic pangram! It uses every letter of the English alphabet.


Input tokens: 10
Output tokens: 17
Total tokens: 27


In [2]:
import pandas as pd
from google import genai
import time
import sys

# Initialize the Gemini client
client = genai.Client()

# Read the Excel file
df = pd.read_excel('gdpval_data.xlsx')

# Sort by sector
df_sorted = df.sort_values(by='sector').reset_index(drop=True)

# Create new columns for output and token counts if they don't exist
if 'output' not in df_sorted.columns:
    df_sorted['output'] = ''
if 'input_tokens' not in df_sorted.columns:
    df_sorted['input_tokens'] = 0
if 'output_tokens' not in df_sorted.columns:
    df_sorted['output_tokens'] = 0

# Process only the first 50 rows (after sorting)
num_rows_to_process = 50
max_retries = 3

for index, row in df_sorted.head(num_rows_to_process).iterrows():
    prompt = row['prompt']
    retry_count = 0
    success = False
    
    while retry_count < max_retries and not success:
        try:
            # Generate content using Gemini
            response = client.models.generate_content(
                model="gemini-2.0-flash", 
                contents=prompt
            )
            
            # Store the output and token counts
            df_sorted.at[index, 'output'] = response.text
            df_sorted.at[index, 'input_tokens'] = response.usage_metadata.prompt_token_count
            df_sorted.at[index, 'output_tokens'] = response.usage_metadata.candidates_token_count
            
            print(f"Processed row {index + 1}/{num_rows_to_process} (Sector: {row['sector']})", flush=True)
            success = True
            
            # Optional: Add a small delay to avoid rate limiting
            time.sleep(1.5)
            
        except Exception as e:
            retry_count += 1
            if retry_count < max_retries:
                print(f"Error processing row {index + 1}: {str(e)} - Retrying ({retry_count}/{max_retries})...", flush=True)
                time.sleep(2)  # Wait a bit longer before retrying
            else:
                print(f"Error processing row {index + 1} after {max_retries} attempts: {str(e)}", flush=True)
                df_sorted.at[index, 'output'] = f"ERROR: {str(e)}"
                df_sorted.at[index, 'input_tokens'] = 0
                df_sorted.at[index, 'output_tokens'] = 0

# Save to a new Excel file
output_filename = 'gdpval_data_first50_by_sector.xlsx'
df_sorted.to_excel(output_filename, index=False)
print(f"\nProcessing complete! Results saved to {output_filename}", flush=True)
print(f"Total rows processed: {num_rows_to_process}", flush=True)

Processed row 1/50 (Sector: Finance and Insurance)
Processed row 2/50 (Sector: Finance and Insurance)
Processed row 2/50 (Sector: Finance and Insurance)
Processed row 3/50 (Sector: Finance and Insurance)
Processed row 3/50 (Sector: Finance and Insurance)
Error processing row 4: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 200\nPlease retry in 697.862698ms.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'q

KeyboardInterrupt: 